# go-goroute详解

goroutine 使用的内存比线程更少，Go 语言运行时会自动在配置的一组逻辑处理器上调度执行 goroutine。

通道（channel）是一种内置的数据结构，可以让用户在不同的 goroutine 之间同步发送具有类型的消息。

使用关键字 go 创建 goroutine 来运行函数。

并发（concurrency）不是并行（parallelism）。

并行是让不同的代码片段同时在不同的物理处理器上执行。并行的关键是同时做很多事情，
而并发是指同时管理很多事情，这些事情可能只做了一半就被暂停去做别的事情了。

在很多情况下，并发的效果比并行好，因为操作系统和硬件的总资源一般很少，但能支持系统同时做很多事情。这种“使用较少的资源做更多的事情”的哲学，也是指导 Go 语言设计的哲学。

如果希望让 goroutine **并行**，必须使用多于一个逻辑处理器。当有多个逻辑处理器时，调度器会将 goroutine 平等分配到每个逻辑处理器上。这会让 goroutine 在不同的线程上运行。

不过要想真的实现并行的效果，用户需要让自己的程序运行在**有多个物理处理器的机器**上。否则，哪怕 Go 语言运行时使用多个线程，goroutine 依然会在同一个物理处理器上并发运行，达不到并行的效果。

如果两个或者多个 goroutine 在没有互相同步的情况下，访问某个共享的资源，并试图同时读和写这个资源，就处于相互竞争的状态，这种情况被称作竞争状态（race candition）。

### 通道

使用通道，通过发送和接收需要共享的资源，在 goroutine 之间做同步。

在 Go 语言中需要使用内置函数 make 来创建一个通道, make 的第一个参数需要是关键字 chan，之后跟着允许通道交换的数据的类型。

如果创建的是一个有缓冲的通道，之后还需要在第二个参数指定这个通道的缓冲区的大小。

// 无缓冲的整型通道
`unbuffered := make(chan int)`
// 有缓冲的字符串通道
`buffered := make(chan string, 10)`

向通道发送值或者指针需要用到<-操作符

// 通过通道发送一个字符串
`buffered <- "Gopher"`

// 从通道接收一个字符串
`value := <-buffered`

### 无缓冲通道

无缓冲的通道（unbuffered channel）是指在接收前没有能力保存任何值的通道。这种类型的通道要求发送 goroutine 和接收 goroutine 同时准备好，才能完成发送和接收操作。如果两个 goroutine 没有同时准备好，通道会导致先执行发送或接收操作的 goroutine 阻塞等待。这种对通道进行发送和接收的交互行为本身就是同步的。其中任意一个操作都无法离开另一个操作单独存在。

### 有缓冲通道

有缓冲的通道（buffered channel）是一种在被接收前能存储一个或者多个值的通道。这种类型的通道并不强制要求 goroutine 之间必须同时完成发送和接收。通道会阻塞发送和接收动作的条件也会不同。只有在通道中没有要接收的值时，接收动作才会阻塞。只有在通道没有可用缓冲区容纳被发送的值时，发送动作才会阻塞。这导致有缓冲的通道和无缓冲的通道之间的一个很大的不同：无缓冲的通道把保证进行发送和接收的 goroutine 会在同一时间进行数据交换；有缓冲的通道没有这种保证。

### 小结

* 可以使用通道来控制程序的生命周期。
* 带 default 分支的 select 语句可以用来尝试向通道发送或者接收数据，而不会阻塞。
* 有缓冲的通道可以用来管理一组可复用的资源。
* 语言运行时会处理好通道的协作和同步。
* 使用无缓冲的通道来创建完成工作的 goroutine 池。
* 任何时间都可以用无缓冲的通道来让两个 goroutine 交换数据，在通道操作完成时一定保证对方接收到了数据